In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os, uuid
from sklearn.pipeline import Pipeline
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import BytesIO
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder
warnings.filterwarnings('ignore')
from pycaret.classification import *
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import average_precision_score
sns.set()

In [2]:
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client("data")

In [3]:
def download_blob_to_df(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    download_stream = blob_client.download_blob()
    blob_data = BytesIO(download_stream.readall())
    return pd.read_csv(blob_data)

In [4]:
in_time_df = download_blob_to_df("in_time.csv")
manager_survey_data_df = download_blob_to_df("manager_survey_data.csv")
employee_survey_data_df = download_blob_to_df("employee_survey_data.csv")
out_time_df = download_blob_to_df("out_time.csv")
general_data_df = download_blob_to_df("general_data.csv")

In [5]:
class mergeDataFrame (BaseEstimator, TransformerMixin):
    def __init__(self, employee_survey_data_df, manager_survey_data_df, in_time_df, out_time_df):
        self.employee_survey_data_df = employee_survey_data_df
        self.manager_survey_data_df = manager_survey_data_df
        self.in_time_df = in_time_df
        self.out_time_df = out_time_df
        
            
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.join(self.employee_survey_data_df.set_index('EmployeeID'), on='EmployeeID')
        X = X.join(self.manager_survey_data_df.set_index('EmployeeID'), on='EmployeeID')

        employee_id_index = self.in_time_df['Unnamed: 0']
        self.in_time_df.fillna(0,inplace=True)
        self.out_time_df.fillna(0,inplace= True)
        self.in_time_df.drop(columns={'Unnamed: 0':'EmployeeID'}, inplace=True)
        self.out_time_df.drop(columns={'Unnamed: 0':'EmployeeID'}, inplace=True)
        

        for col in self.in_time_df.columns:
            self.in_time_df[col] = pd.to_datetime(self.in_time_df[col], errors='coerce')

        for col in self.out_time_df.columns:
            self.out_time_df[col] = pd.to_datetime(self.out_time_df[col], errors='coerce')
        
        daily_hours = (self.out_time_df - self.in_time_df).applymap(lambda x: x.total_seconds() / 3600)
        daily_hours = daily_hours.fillna(0)
        daily_hours['PresenceIndicator'] = daily_hours.iloc[:, 1:].apply(lambda row: sum(1 if hours >= 8 else 0 for hours in row), axis=1)

        presence_indicator = pd.DataFrame({'EmployeeID': employee_id_index, 'PresenceIndicator': daily_hours['PresenceIndicator']})
        
        X = X.join(presence_indicator.set_index('EmployeeID'), on='EmployeeID', how='inner')
        return X
        

In [6]:
class deleteColumns(BaseEstimator, TransformerMixin):
    def __init__(self, array):
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.drop(columns=self.array, inplace=True)
        return X

In [7]:
class encodingData (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X["Attrition"] = X["Attrition"].map({"Yes": 1, "No": 0})
        X["BusinessTravel"] = X["BusinessTravel"].map({"Non-Travel": 0, "Travel_Rarely": 1, "Travel_Frequently": 2})
        ordinal_encoder = OrdinalEncoder()
        for i in X.select_dtypes(include=["object"]).keys():
            X[i] = ordinal_encoder.fit_transform(X[[i]])
        return X

In [8]:
class cleanData (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        environmentSatisfactionMedian = X.EnvironmentSatisfaction.median()
        jobSatisfactionMedian = X.JobSatisfaction.median()
        workLifeBalanceMedian = X.WorkLifeBalance.median()
        totalWorkingYears_median = X['TotalWorkingYears'].median()
        X['EnvironmentSatisfaction'].fillna(environmentSatisfactionMedian, inplace = True)
        X['JobSatisfaction'].fillna(jobSatisfactionMedian, inplace = True)
        X['WorkLifeBalance'].fillna(workLifeBalanceMedian, inplace = True)
        X['TotalWorkingYears'].fillna(totalWorkingYears_median, inplace = True)
        X['NumCompaniesWorked'].fillna(1.0, inplace = True)
        X = X.fillna(0)
        return X

In [9]:
class corrData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Calculate correlations and retain only significant ones
        corr_x = self.retain_terminal(X.corr())
        significant_parameters, _ = self.separation_significant_parameters(corr_x)

        # Recalculate correlations only among significant parameters
        corr_tmp = X[significant_parameters].corr()
        corr_tmp = self.retain_terminal(corr_tmp)

        # Select only features with non-zero correlation to 'Attrition'
        significant_features = corr_tmp.Attrition[corr_tmp.Attrition != 0].index.tolist()
        return X[significant_features]

    def retain_terminal(self, frame):
        # Set correlation values below threshold to zero
        for i in frame.columns:
            for j in frame.index:
                if abs(frame.loc[j, i]) < 0.1:
                    frame.loc[j, i] = 0
        return frame

    def separation_significant_parameters(self, frame):
        # Separate parameters based on their significance
        significant_parameter = []
        insignificant_parameter = []
        for column in frame.columns:
            if not all(frame[column] == 0):
                significant_parameter.append(column)
            else:
                insignificant_parameter.append(column)
        return significant_parameter, insignificant_parameter


In [10]:
pipeline = Pipeline([
    ('merge', mergeDataFrame(employee_survey_data_df, manager_survey_data_df, in_time_df, out_time_df)),
    ('delete', deleteColumns(['EmployeeID', 'EmployeeCount', 'Over18', 'StandardHours', 'MaritalStatus', 'Gender', 'Age'])),
    ('encoding', encodingData()),
    ('clean', cleanData()),
    ('corr', corrData())
])
from sklearn import set_config
set_config(display='diagram')
display(pipeline)

Pipeline(steps=[('merge',
                 mergeDataFrame(employee_survey_data_df=      EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
0              1                      3.0              4.0              2.0
1              2                      3.0              2.0              4.0
2              3                      2.0              2.0              1.0
3              4                      4.0              4.0              3.0
4              5                      4.0              1.0              3.0
...          ...                      ...              ...              ...
4405        4406                      4.0              1.0              3.0
4406        4407                      4.0              4.0              3.0
4407        4408                      1.0              3.0              3.0
4408        4409                      4.0              1.0              3.0
4409        4410                      1.0              3.0              NaN

[4410 rows x 4 columns],
                                in_time_df=      Unnamed: 0  2015-01-01           2015-01-02           2015-01-...
3     2015-12-31 17:09:14  
4     2015-12-31 17:42:14  
...                   ...  
4405  2015-12-31 18:30:41  
4406  2015-12-31 16:18:39  
4407  2015-12-31 18:08:55  
4408  2015-12-31 19:33:45  
4409  2015-12-31 16:39:18  

[4410 rows x 262 columns])),
                ('delete',
                 deleteColumns(array=['EmployeeID', 'EmployeeCount', 'Over18',
                                      'StandardHours', 'MaritalStatus',
                                      'Gender', 'Age'])),
                ('encoding', encodingData()), ('clean', cleanData()),
                ('corr', corrData())])

In [11]:
dataset = pipeline.fit_transform(general_data_df)

labels = dataset.keys().to_list()
labels.remove('Attrition')

X = dataset[labels]
y = dataset['Attrition']
# Features
X



,BusinessTravel,TotalWorkingYears,YearsAtCompany,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,PresenceIndicator
0,1,1.0,1,0,3.0,4.0,0
1,2,6.0,5,4,3.0,2.0,42
2,2,5.0,5,3,2.0,2.0,0
3,0,13.0,8,5,4.0,4.0,0
4,1,9.0,6,4,4.0,1.0,115
...,...,...,...,...,...,...,...
4405,1,10.0,3,2,4.0,1.0,237
4406,1,10.0,3,2,4.0,4.0,0
4407,1,5.0,4,2,1.0,3.0,41
4408,1,10.0,9,8,4.0,1.0,241


# Choix des métriques de performance 

Afin de mesurer la performance de nos modèles pour savoir quels seront les meilleurs modèles, il est important de définir en amont des métriques de performance pour effectuer ce choix. Pour ce faire il existe plusieurs métriques de performance pour les problèmes de classification binaire.

### Accuracy

L'accuracy est la métrique la plus simple et la plus utilisée pour mesurer la performance d'un modèle. Elle représente le nombre de prédictions correctes (à la fois positives et négatives) par rapport au nombre total de prédictions. Cependant, cette métrique n'est pas toujours la meilleure pour mesurer la performance d'un modèle. En effet, dans le cas où les classes ne sont pas équilibrées, l'accuracy peut être trompeuse. Par exemple, si 90% des données appartiennent à la classe 1 et 10% à la classe 0, un modèle qui prédit toujours la classe 1 aura une accuracy de 90%.

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

## Precision

La precision est une métrique qui mesure la proportion de vrais positifs parmi les prédictions positives. Elle est utile lorsque le coût des faux positifs est élevé. Par exemple, dans le cas d'un modèle qui prédit si un email est un spam ou non, il est préférable d'avoir un faible taux de faux positifs (emails légitimes classés comme spam) même si cela signifie que certains spams seront classés comme légitimes.

$$
\text{Précision} = \frac{TP}{TP + FP}
$$

## Recall 

Le recall (ou rappel) est une métrique qui mesure la proportion de vrais positifs parmi les vrais positifs et les faux négatifs. Elle est utile lorsque le coût des faux négatifs est élevé. Par exemple, dans le cas d'un modèle qui prédit si un patient a une maladie ou non, il est préférable d'avoir un faible taux de faux négatifs (patients malades classés comme sains) même si cela signifie que certains patients sains seront classés comme malades.

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

## F1 Score

Le F1 score est la moyenne harmonique de la precision et du recall. Il est utile lorsque les classes sont déséquilibrées. En effet, dans ce cas, l'accuracy n'est pas une bonne métrique de performance. Par exemple, si 90% des données appartiennent à la classe 1 et 10% à la classe 0, un modèle qui prédit toujours la classe 1 aura une accuracy de 90%. Cependant, le F1 score de ce modèle sera de 0. 

$$
F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}
$$

## AUC-ROC 

L'AUC-ROC (Area Under the Receiver Operating Characteristic Curve) est la mesure de la capacité d'un modèle à distinguer entre les classes. ROC est une courbe de probabilité qui trace le taux de vrais positifs contre le taux de faux positifs à différents seuils de classification. 

Un modèle qui fait des prédictions parfaites aurait une AUC de 1, tandis qu'un modèle qui fait des prédictions aléatoires aurait une AUC de 0.5.

L'AUC-ROC est particulièrement utile lorsque les classes sont déséquilibrées. Elle est insensible au déséquilibre des classes et se concentre sur la capacité du modèle à distinguer entre les classes. Cependant, si le coût des faux positifs est très élevé, il peut être trompeur.

## AUC PR

L'AUC-PR est similaire à l'AUC-ROC mais se concentre sur la relation entre la précision (proportion de vrai positifs parmis les prédiction positives) et le recall (proportion de vrais positifs parmis les vrais cas positifs). 

Dans un contexte où les positifs (comme les spams dans un exemple de filtrage) sont rares, l'AUC-PR donne une meilleure indication de la performance du modèle. 

L'AUC-PR est préférable lorsque le déséquilibre des classe est un problème et que l'on s'intéresse davantage à la performance du modèle sur la classe minoritaire. Il est plus informatif que l'AUC-ROC dans les cas où les positifs sont beaucoup moins fréquents que les négatifs.

## Choix appliqué au contexte 

Pour le problème de l'attrition des employés, qui est un problème de classification binaire (les employés quittent ou ne quittent pas l'entreprise) 

Il est important de prendre en compte le coût relatif des erreurs (faux positifs vs faux négatifs) et la distribution de la classe (équilibrée ou déséquilibrée).

* Si le *coût d'un faux négatif* (ne pas identifier un employé qui est susceptible de quitter) est élevée car cela pourrait perturber les opérations de l'entreprise, entaîner la perte de talent clés ou nuire à la plannification des ressources, alors il est préférable de privilégier *le recall*.

* Si le coût d'un faux positif (croire à tort qu'un employé va partir) est élevée, ce qui conduit à un problème éthique et conduire à des dépenses inutiles en interventions de rétention ou à une ambiance de méfiance, alors il est préférable de privilégier la précision. 

Dans le contexte de notre problème, ces deux types d'erreurs sont importants, le F1-Score peut être un bon choix car il équilibre la précision et le recall. Cependant nos données étant déséquilibrées, les mesures telles que la précision, le recall et le F1-Score peuvent ne pas refléter fidèlement la performance de notre modèle.

Dans ce cas L'AUC-ROC ou l'AUC-PR pourraient être plus appropriées. L'AUC-ROC est moins sensible aux déséquilibres de classe, mais si la classe positive (les employés qui quittent est beaucoup plus petite, l'AUC-PR peut être préférable car elle se concentre sur la classe minoritaire. 

Les autres métriques sont évidemment utiles pour évaluer la performance des modèles, mais prendrons en priorité l'AUC-PR et le F1-Score pour évaluer la performance de nos modèles.

# Choix du modèle 

Afin de choisir le modèle le plus adapté à notre problème de classification, nous allons comparer les performances de plusieurs modèles de classification. Pour réaliser cela, nous allons utiliser la bibliothèque pycaret qui permet de comparer les performances de plusieurs modèles de classification. Cette bibliothèque va lancer un processus d'entraînement et de test sur plusieurs modèles de classification et nous donnera les performances de chacun de ces modèles.

La métrique de l'AUC-PR n'étant pas nativement disponible dans la bibliothèque pycaret. Toutefois il est possible de créer une métrique personnalisée pour l'AUC-PR. 

Pour cela, nous allons utiliser 

In [12]:
def auc_pr_score(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

In [13]:
clf = setup(data=dataset, target='Attrition')
add_metric('AUC-PR', 'AUC-PR', auc_pr_score, greater_is_better=True)
best__models = compare_models(sort='AUC-PR')

results = pull()

results

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:00:34
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC-PR,TT (Sec)
et,Extra Trees Classifier,0.9657,0.9705,0.8436,0.9381,0.8876,0.8675,0.8696,0.9035,0.033
rf,Random Forest Classifier,0.9521,0.9630,0.7692,0.9225,0.8377,0.8099,0.8152,0.8645,0.042
dt,Decision Tree Classifier,0.9219,0.8691,0.7552,0.7640,0.7573,0.7109,0.7124,0.7793,0.005
lightgbm,Light Gradient Boosting Machine,0.9067,0.9364,0.5240,0.8402,0.6419,0.5920,0.6155,0.7205,0.138
dummy,Dummy Classifier,0.8387,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5807,0.005
gbc,Gradient Boosting Classifier,0.8698,0.8385,0.3113,0.7323,0.4328,0.3730,0.4185,0.5774,0.033
ada,Ada Boost Classifier,0.8623,0.8085,0.2692,0.6880,0.3814,0.3219,0.3690,0.5375,0.018
nb,Naive Bayes,0.8549,0.7575,0.2891,0.6118,0.3892,0.3192,0.3496,0.5078,0.005
qda,Quadratic Discriminant Analysis,0.8510,0.7527,0.2951,0.5796,0.3879,0.3137,0.3382,0.4942,0.005
lr,Logistic Regression,0.8523,0.7692,0.1706,0.6526,0.2674,0.2173,0.2788,0.4785,0.384


In [20]:
def optimize_model(model):
    return tune_model(model, optimize='AUC-PR')

In [21]:
# Extra Trees Classifier with sci-kit learn

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, average_precision_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = ExtraTreesClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(average_precision_score(y_test, y_pred))

# Print auc-pr score
print(auc_pr_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1115
           1       0.95      0.85      0.89       208

    accuracy                           0.97      1323
   macro avg       0.96      0.92      0.94      1323
weighted avg       0.97      0.97      0.97      1323

[[1105   10]
 [  32  176]]
0.9682539682539683
0.8248491566463917
0.9082889290262562


In [23]:
def optimize_with_grid_search_cv(model, param_grid):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=auc_pr_scorer, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_, grid_search.best_score_

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:39:32
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC-PR
Fold,,,,,,,,
0,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
1,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
2,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
3,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
4,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
5,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
6,0.1618,0.5,1.0,0.1618,0.2786,0.0,0.0,0.5809
7,0.1623,0.5,1.0,0.1623,0.2793,0.0,0.0,0.5812
8,0.1591,0.5,1.0,0.1591,0.2745,0.0,0.0,0.5795


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, average_precision_score
auc_pr_scorer = make_scorer(auc_pr_score , needs_proba=True)
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=auc_pr_scorer, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 20,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [16]:
grid_search.best_score_

0.9014246049937386

In [17]:
# Random Forest Classifier with sci-kit learn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, average_precision_score


model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(average_precision_score(y_test, y_pred))
print(auc_pr_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1115
           1       0.92      0.75      0.83       208

    accuracy                           0.95      1323
   macro avg       0.94      0.87      0.90      1323
weighted avg       0.95      0.95      0.95      1323

[[1101   14]
 [  51  157]]
0.9508692365835223
0.7315593241345121
0.8557425500564598


In [18]:
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=auc_pr_scorer, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [19]:
grid_search.best_score_

0.8840096620134421